## Importing the libararies

In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import scipy.spatial
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from myimg import *
from mymodel import CSRNet
import torch
%matplotlib inline

In [2]:
from helping_fns import *
from mydataset import *

## Standard transforms to apply

In [3]:
from torchvision import datasets, transforms
trans=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [4]:
img_paths = []
val_paths='shanghaitech_with_people_density_map/ShanghaiTech/part_B/test_data/images/'
for img_path in glob.glob(os.path.join(val_paths, '*.jpg')):
    img_paths.append(img_path)

In [5]:
val_list=img_paths
val_list=listDataset(val_list,shuffle=True,transform=trans)
val_loader=torch.utils.data.DataLoader(val_list,batch_size=1,shuffle=True)

In [6]:
test_lod=next(iter(val_loader))
test_lod[0].shape,test_lod[1].sum()

(torch.Size([1, 3, 768, 1024]), tensor(180.9997))

## Model to validate on

In [7]:
model = CSRNet()
model = model.cuda()
checkpoint = torch.load('MODEL_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [9]:
mae = 0
mae_t=0
for i,(img,groundtruth) in enumerate(val_loader):

    img = img.cuda()
   
    output = model(img)
    print('PREDICTED-COUNT------',output.detach().sum())
    print('ORIGINAL-COUNT------',groundtruth.sum())
#    show_tensor_images(output.detach())
    mae = abs(output.detach().cpu().sum().numpy()-groundtruth.sum().numpy())
    mae_t+=mae
    print('MAE:')
    print(i,mae)
print(mae_t/len(img_paths))

PREDICTED-COUNT------ tensor(106.7289, device='cuda:0')
ORIGINAL-COUNT------ tensor(85.0164)
MAE:
0 21.712471
PREDICTED-COUNT------ tensor(199.2528, device='cuda:0')
ORIGINAL-COUNT------ tensor(206.8907)
MAE:
1 7.637909
PREDICTED-COUNT------ tensor(119.0334, device='cuda:0')
ORIGINAL-COUNT------ tensor(86.5303)
MAE:
2 32.503105
PREDICTED-COUNT------ tensor(70.9757, device='cuda:0')
ORIGINAL-COUNT------ tensor(47.5666)
MAE:
3 23.40908
PREDICTED-COUNT------ tensor(97.5442, device='cuda:0')
ORIGINAL-COUNT------ tensor(63.7968)
MAE:
4 33.747437
PREDICTED-COUNT------ tensor(83.6122, device='cuda:0')
ORIGINAL-COUNT------ tensor(86.4671)
MAE:
5 2.8548584
PREDICTED-COUNT------ tensor(60.0093, device='cuda:0')
ORIGINAL-COUNT------ tensor(44.2064)
MAE:
6 15.802856
PREDICTED-COUNT------ tensor(72.8246, device='cuda:0')
ORIGINAL-COUNT------ tensor(31.1993)
MAE:
7 41.62532
PREDICTED-COUNT------ tensor(158.2156, device='cuda:0')
ORIGINAL-COUNT------ tensor(184.7536)
MAE:
8 26.537964
PREDICTED-COUNT-